In [1]:
import numpy as np
import BioSimSpace as BSS


Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



/home/jguven/Software/miniconda3/envs/bss22/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/bss22/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/jguven/Software/miniconda3/envs/bss22/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/bss22/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [4]:
# Paths
path_to_ligands = "../../BioSimSpaceTutorials/04_fep/inputs/ligands/"
path_to_protein = "inputs/protein/tyk2_clean.pdb"

In [5]:
# Load input files
ligand_1 = BSS.IO.readMolecules(path_to_ligands+"ejm_50.mol2")[0]
ligand_2 = BSS.IO.readMolecules(path_to_ligands+"ejm_49.mol2")[0]
protein = BSS.IO.readMolecules(path_to_protein)[0]
protein

<BioSimSpace.Molecule: nAtoms=4666, nResidues=289>

# Parameterising input molecules

In [6]:
ligand_1 = BSS.Parameters.gaff2(ligand_1).getMolecule()

In [7]:
ligand_2 = BSS.Parameters.gaff2(ligand_2).getMolecule()

# NOTE

Before the below step, run `pdb4amber` to first remove hydrogens and then to add all missing atoms. Details on Notion.

In [8]:
protein = BSS.Parameters.parameterise(protein, forcefield="ff14SB").getMolecule()

# Alignment and merging

To transform two ligands, they need to be well aligned. In BSS, we can do this with the function `Align`. It uses a Maximum Common Substructure (MCS) search, which finds mappings between atom indices in the two molecules. 

`BSS.Align.matchAtoms()` matches the atoms with MCS. It finds mappings between atom indices in molecule 1 and molecule 2. 

`BSS.Align.rmsdAlign()` uses the above mapping to actually align the atoms from molecule 1 to molecule 2. 

In [9]:
atom_mapping = BSS.Align.matchAtoms(ligand_1, ligand_2)
aligned_ligand_1 = BSS.Align.rmsdAlign(ligand_1, ligand_2, atom_mapping)

Now we have to create a 'merged' molecule, i.e. a molecule that we can transform in a way such that the endpoints are both input ligands:

In [10]:
merged = BSS.Align.merge(ligand_1, ligand_2)

Add the merged ligand to the protein to create the system:

In [11]:
system = merged + protein

# Solvation

We will have two environments:
1. Each ligand (`merged`) in the solvent, **not bound** to the protein, call this `unbound`
2. Each ligand (`system`) in the solvent, **bound** to the protein, call this `bound`.


In [12]:
box_dimensions =  3 * [10 * BSS.Units.Length.nanometer] 
unbound = BSS.Solvent.tip3p(molecule=merged, box=box_dimensions)
bound = BSS.Solvent.tip3p(molecule=system, box=box_dimensions)

# FEP Protocol

Now we can write the protocol and the set up files and soon finally run the FEP.  

In [13]:
BSS.FreeEnergy.engines()

['Somd', 'Gromacs']

In [14]:
protocol = BSS.Protocol.FreeEnergy()
free_energy_ub = BSS.FreeEnergy.Relative(unbound, protocol, engine="Gromacs", work_dir="output/gromacs/ejm50~ejm49/unbound/")
free_energy_b = BSS.FreeEnergy.Relative(bound, protocol, engine="Gromacs", work_dir="output/gromacs/ejm50~ejm49/bound/")

# Setup folders etc for running on GPUs

See Notion.